### 加载模型和数据集

In [1]:
import torch

print(torch.cuda.is_available())

from modelscope import AutoTokenizer
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq

tokenizer = AutoTokenizer.from_pretrained("./Qwen2.5-0.5B-Instruct", use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("./Qwen2.5-0.5B-Instruct", device_map="cuda", torch_dtype="auto")

True


d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
from datasets import Dataset
from swanlab.integration.huggingface import SwanLabCallback
from peft import LoraConfig, TaskType, get_peft_model
import swanlab

C:\Users\29277\AppData\Local\Temp\ipykernel_14572\2019269277.py:3: DeprecationWarning: The module 'huggingface' is deprecated and will be removed in future versions. Please update your imports to use 'transformers' instead.
  from swanlab.integration.huggingface import SwanLabCallback


In [3]:
train_df = pd.read_json('binary-class.json')
train_ds = Dataset.from_pandas(train_df)

In [4]:
train_df

,文本,风险类别
0,您对移民有兴趣吗？我们提供专业的移民咨询和投资方案，确保您的资金安全并助您顺利移民。我们的服...,无风险
1,尊敬的用户，您可以通过我们的官方应用程序随时查看您的营养账户余额。只需登录并访问您的个人账户...,无风险
2,为了确保您能够顺利接收剧院的最新演出信息和票务通知，请验证您的手机号码。点击以下链接完成验证...,无风险
3,尊敬的信用卡用户，您好！为了提供更便捷的服务，我们特别推出了一项新的信用卡还款优惠计划。只需...,有风险
4,在赫兹APP上交友是一种很好的社交方式，但请谨慎对待任何涉及金钱的提议，确保安全第一。,无风险
...,...,...
29995,我们注意到您的信用卡账户近期存在异常交易，为了您的账户安全，请立即设置新的安全问题。请点击以...,有风险
29996,为了确保您的旅游咨询顺利进行，请您填写以下信息以便我们为您提供更精准的服务：https://...,有风险
29997,您好，为了提供更好的购物体验，我们平台推出了预约服务。您可以通过点击以下链接预约您心仪的商品...,无风险
29998,您好，为了确保您的包裹能够安全送达，请输入以下验证码以确认您的身份：123456。请在收到后...,无风险


In [5]:
s = [(len(i),i) for i in list(train_ds['文本'])]
s.sort(reverse=True)

In [6]:
s

[(205,
  '您好，我们是一家知名的投资公司，正在寻找有潜力的合作伙伴。我们注意到您在职业招聘网站上的简历，认为您具备成为我们团队一员的潜力。为了启动合作，请您先支付一笔小额资金作为项目启动费。请通过以下账户进行转账：账户名：李四，账号：098765432109，开户行：工商银行。转账完成后，请将转账凭证发送至我们的邮箱：invest@example.com。我们将在确认收到款项后，立即为您安排合作事宜。期待与您的合作！'),
 (192,
  '恭喜您，您的汽车改装项目已通过我们的初步审核。为了确保项目顺利进行，请您按照以下步骤操作：首先，您需要通过银行转账的方式预付部分改装费用至我们的官方账户。账户信息如下：账户名：汽车改装中心，账号：1234567890，开户行：中国工商银行。转账后，请将转账凭证发送至我们的客服邮箱：service@qichegaizhuang.com，以便我们确认并安排后续工作。感谢您的支持与合作！'),
 (192,
  '尊敬的用户，您好！我们注意到您最近在保险理财产品上的兴趣，我们很高兴地通知您，您已被选中参与一项特殊的保险投资计划。为了确保您的权益，请按照我们提供的合同模板签订电子合同，并通过以下链接提交：https://secure-contract-sign.com/login。合同签订后，您将立即享受我们提供的优惠利率和额外保障。请在48小时内完成操作，以免错过此次机会。期待您的积极参与！'),
 (191,
  '我们注意到您对健康饮食的关注，特别为您提供一项独家营养咨询服务。只需预约并支付一小笔定金，您将获得由资深营养师提供的个性化饮食计划。此外，我们还提供一项投资机会，您可以通过参与我们的健康产品推广计划获得额外收益。详情请访问我们的网站：https://healthnutrition.com/invest，预约咨询请发送邮件至 service@healthnutrition.com。'),
 (189,
  '您好，由于我们的系统错误，您的剧院票务订单未能成功出票。为了尽快解决这个问题，请您按照以下步骤操作：首先，点击此链接https://theater-support.com/issue-resolve，然后按照页面提示完成身份验证。随后，您需要向我们指定的账户转账100元作为临时保证金，以确保您的票务问题能够

In [3]:
prompt = '''在这个任务中，你是一位资深的反诈骗网络安全分析师，你的职责是利用你的专业知识和对网络诈骗行为的深刻理解，从短信文本中识别出可能存在的欺诈行为和风险类别。你的工作对于提前预警潜在的网络诈骗，保护用户财产安全和个人信息不被侵犯具有重要意义。现在，请仔细审查以下短信文本，并运用你的专业判断该短信是否有风险，回答“无风险”或“有风险”'''

def process_func(example):
    """
    将数据集进行预处理
    """
    MAX_LENGTH = 384 
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(
        f"<|im_start|>system\n{prompt}<|im_end|>\
        \n<|im_start|>user\n{example['文本']}<|im_end|>\n<|im_start|>assistant\n",
        add_special_tokens=False,
    )
    response = tokenizer(f"{example['风险类别']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = (
        instruction["attention_mask"] + response["attention_mask"] + [1]
    )
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}  


In [8]:
train_dataset = train_ds.map(process_func, remove_columns=train_ds.column_names)
train_dataset.save_to_disk("binary-class")

d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]
Saving the dataset (1/1 shards): 100%|██████████| 30000/30000 [00:00<00:00, 499845.55 examples/s]


In [4]:
from datasets import load_from_disk
train_dataset = load_from_disk("binary-class")

In [5]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 30000
})

### 模型微调

In [30]:
model = AutoModelForCausalLM.from_pretrained("./Qwen2.5-0.5B-Instruct", device_map="auto", torch_dtype="auto")
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "lm_head"],
    inference_mode=False,  # 训练模式
    r=8,  # Lora 秩
    lora_alpha=32,  # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1,  # Dropout 比例
)

# for param in model.parameters():
#     param.requires_grad = True
    
model = get_peft_model(model, config)

In [31]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
   

In [32]:
model.print_trainable_parameters()

trainable params: 5,621,760 || all params: 499,654,528 || trainable%: 1.1251


In [33]:
args = TrainingArguments(
    output_dir="./output/binary-class0113",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=2000,
    num_train_epochs=6,
    save_steps=2000,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True,
    report_to="none",
)

swanlab_callback = SwanLabCallback(
    project="Qwen2.5-0.5B-Instruct-Finetuning",
    experiment_name="Qwen2.5-0.5B-Instruct-binary-class0113",
    description="Qwen2.5-0.5B-Instruct模型在binary-class数据集上微调。",
    config={
        "model": "qwen/Qwen2.5-0.5B-Instruct",
        "dataset": "FGRC-SCD电信诈骗数据集 binary-class",
    }
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    callbacks=[swanlab_callback],
)

In [34]:
trainer.train()

swanlab: swanlab version 0.4.2 is available!  Upgrade: `pip install -U swanlab`
swanlab: Tracking run with swanlab version 0.3.21                                  
swanlab: Run data will be saved locally in d:\fudan\LLM\FineTune-Qwen\swanlog\run-20250114_004416-a3b1799d
swanlab: 👋 Hi tangerine, welcome to swanlab!
swanlab: Syncing run Qwen2.5-0.5B-Instruct-binary-class0113 to the cloud
swanlab: 🌟 Run `swanlab watch d:\fudan\LLM\FineTune-Qwen\swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@tangerine/Qwen2.5-0.5B-Instruct-Finetuning
swanlab: 🚀 View run at https://swanlab.cn/@tangerine/Qwen2.5-0.5B-Instruct-Finetuning/runs/oazurffjp7apx30j3vy7w


                                                         
 18%|█▊        | 2000/11250 [2:24:33<12:01:25,  4.68s/it]d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\peft\utils\save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


{'loss': 0.2481, 'grad_norm': 10.325764656066895, 'learning_rate': 8.222222222222222e-05, 'epoch': 1.07}


d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\torch\utils\checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\swanlab\api\http.py:114: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  if (self.sid_expired_at - datetime.utcnow()).

{'loss': 0.1275, 'grad_norm': 8.885976791381836, 'learning_rate': 6.444444444444446e-05, 'epoch': 2.13}


d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\torch\utils\checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\swanlab\api\http.py:114: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  if (self.sid_expired_at - datetime.utcnow()).

{'loss': 0.1218, 'grad_norm': 3.727365016937256, 'learning_rate': 4.666666666666667e-05, 'epoch': 3.2}


d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\torch\utils\checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\swanlab\api\http.py:114: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  if (self.sid_expired_at - datetime.utcnow()).

KeyboardInterrupt: 

In [35]:
swanlab.finish()

swanlab: 🌟 Run `swanlab watch d:\fudan\LLM\FineTune-Qwen\swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@tangerine/Qwen2.5-0.5B-Instruct-Finetuning
swanlab: 🚀 View run at https://swanlab.cn/@tangerine/Qwen2.5-0.5B-Instruct-Finetuning/runs/oazurffjp7apx30j3vy7w
swanlab: \ Updating experiment status...                                           

d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\executing\executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
d:\Programs\anaconda3\envs\cuda121-2\Lib\ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)


ApiError: (409, 'Conflict')